In [8]:
# # Analyse et Prédiction du Diabète Type 2
# Objectif: Prédire l'apparition du T2D dans l'année suivante

# ## 1. Import des librairies et chargement des données

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8') # or 'seaborn-v0_8-whitegrid' if you prefer
sns.set_palette("husl")

print("Librairies importées avec succès")

# ## 2. Chargement et exploration des données

# --- MODIFICATION HERE ---
print("Chargement des données...")

# Define paths (replace with actual paths if different)
TRAINING_DATA_PATH = '../data/training_data.npz'
TRAINING_LABELS_PATH = '../data/training_labels.csv'
EVALUATION_DATA_PATH = '../data/evaluation_data.npz'
SAMPLE_SUBMISSION_PATH = '../data/sample_submission.csv'

# Load training data and feature names
with np.load(TRAINING_DATA_PATH, allow_pickle=True) as f:
    X_train = f["data"]
    feature_names_original = f["feature_labels"]

# Load training labels
training_labels_df = pd.read_csv(TRAINING_LABELS_PATH)
y_train = training_labels_df['Label'].values

# Load evaluation data
with np.load(EVALUATION_DATA_PATH, allow_pickle=True) as f:
    X_eval = f["data"]
    # Assuming evaluation data has the same feature structure,
    # feature_names_original from training can be used if needed for eval feature engineering.

# Load sample submission
sample_submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
# --- END OF MODIFICATION ---

Librairies importées avec succès
Chargement des données...


In [11]:

print(f"Noms des features originales (exemple): {feature_names_original[:5]}") # Print a few original feature names
print(f"Nombre de features originales: {len(feature_names_original)}")
print(f"Données d'entraînement: {X_train.shape}")
print(f"Labels d'entraînement: {y_train.shape}")
print(f"Données d'évaluation: {X_eval.shape}")
print(f"Distribution des classes: {np.bincount(y_train)}")
print(f"Pourcentage de cas positifs: {np.mean(y_train)*100:.2f}%")


Noms des features originales (exemple): ['creatinine' 'urea' 'c_reactive_protein' 'prothrombin_time'
 'total_proteins']
Nombre de features originales: 77
Données d'entraînement: (53652, 12, 77)
Labels d'entraînement: (53652,)
Données d'évaluation: (26826, 12, 77)
Distribution des classes: [50259  3393]
Pourcentage de cas positifs: 6.32%


In [12]:
# Supposons que X_train, feature_names_original, X_eval, y_train sont déjà chargés
# comme dans le script complet que vous avez fourni précédemment.

# Idéalement, la conversion de dtype suivante devrait être faite
# immédiatement après le chargement de X_train et X_eval.
# On le place ici pour le contexte de ce snippet.

if 'X_train' in locals() and X_train.dtype == object:
    print(f"Type de données initial de X_train: {X_train.dtype}")
    print("X_train est de type 'object'. Tentative de conversion en np.float64...")
    try:
        X_train = X_train.astype(np.float64)
        print(f"Conversion de X_train en np.float64 réussie. Nouveau dtype: {X_train.dtype}")
    except ValueError as e:
        print(f"ERREUR: Échec de la conversion de X_train en np.float64: {e}")
        print("Cela peut se produire si l'array contient des chaînes de caractères ou d'autres objets non convertibles en float.")
        print("L'analyse risque d'échouer. Vérifiez le contenu de votre fichier .npz.")
        # Ici, une logique de conversion plus robuste pourrait être nécessaire si l'erreur persiste
        # par exemple, remplacer manuellement des chaînes comme 'NA', 'missing' par np.nan avant astype.

# Assurez-vous que X_eval est aussi converti si vous l'utilisez plus tard avec des opérations numériques.
# if 'X_eval' in locals() and X_eval.dtype == object:
# print(f"Type de données initial de X_eval: {X_eval.dtype}")
# print("X_eval est de type 'object'. Tentative de conversion en np.float64...")
# try:
# X_eval = X_eval.astype(np.float64)
# print(f"Conversion de X_eval en np.float64 réussie. Nouveau dtype: {X_eval.dtype}")
# except ValueError as e:
# print(f"ERREUR: Échec de la conversion de X_eval en np.float64: {e}")


# Vérification des valeurs manquantes et outliers
print("\n=== ANALYSE DE LA QUALITÉ DES DONNÉES ===")
# Ces opérations devraient maintenant fonctionner si X_train est de type numérique
print(f"Valeurs NaN dans X_train: {np.isnan(X_train).sum()}")
print(f"Valeurs infinies dans X_train: {np.isinf(X_train).sum()}")
# Les valeurs négatives peuvent être valides pour certaines features (ex: changement de valeur)
# mais il est bon de vérifier si c'est inattendu pour des résultats de laboratoire typiquement positifs.
# Cette opération nécessite aussi un type numérique.
print(f"Valeurs négatives (potentiellement problématiques): {(X_train < 0).sum() if X_train.dtype != object else 'Non calculé (X_train est object)'}")

# Statistiques par feature
feature_stats = []
# S'assurer que feature_names_original est défini et a la bonne longueur
if 'feature_names_original' not in locals() or len(feature_names_original) != X_train.shape[2]:
    print("Attention: 'feature_names_original' n'est pas défini ou ne correspond pas au nombre de features. Utilisation de noms génériques.")
    feature_names_original = [f"feature_{i}" for i in range(X_train.shape[2])]

for feat_idx in range(X_train.shape[2]):
    # .flatten() crée une copie, donc pas de souci de modification de X_train ici.
    feat_data_all_timesteps = X_train[:, :, feat_idx].flatten()

    # Exclure les NaN pour les calculs de mean, std, min, max
    feat_data_no_nan = feat_data_all_timesteps[~np.isnan(feat_data_all_timesteps)]

    if len(feat_data_no_nan) > 0:
        mean_val = np.mean(feat_data_no_nan)
        std_val = np.std(feat_data_no_nan)
        min_val = np.min(feat_data_no_nan)
        max_val = np.max(feat_data_no_nan)
    else: # Toutes les valeurs pour cette feature sont NaN
        mean_val, std_val, min_val, max_val = np.nan, np.nan, np.nan, np.nan

    stats_dict = {
        'feature_idx': feat_idx,
        'feature_name': feature_names_original[feat_idx],
        'mean': mean_val,
        'std': std_val,
        'min': min_val,
        'max': max_val,
        'nan_rate': np.isnan(feat_data_all_timesteps).mean(), # Taux de NaN sur toutes les valeurs de la feature
        # Pour zero_rate, np.nan == 0 est False, donc les NaNs ne sont pas comptés comme zéros.
        'zero_rate': (feat_data_all_timesteps == 0).mean()
    }
    feature_stats.append(stats_dict)

feature_stats_df = pd.DataFrame(feature_stats)
print("\nStatistiques descriptives par feature (calculées sur les valeurs non-NaN pour mean/std/min/max):")
print(feature_stats_df.head().to_string()) # .to_string() pour un meilleur affichage si les noms sont longs
print(f"\nTaux moyen de valeurs manquantes par feature: {feature_stats_df['nan_rate'].mean():.4f}")
print(f"Taux moyen de zéros par feature: {feature_stats_df['zero_rate'].mean():.4f}")

# Identifier les features avec un taux élevé de NaN
high_nan_threshold = 0.8
high_nan_features = feature_stats_df[feature_stats_df['nan_rate'] > high_nan_threshold]
print(f"\nFeatures avec plus de {high_nan_threshold*100:.0f}% de NaN ({len(high_nan_features)}):")
if not high_nan_features.empty:
    print(high_nan_features[['feature_name', 'nan_rate']].to_string(index=False))
else:
    print(f"Aucune feature n'a plus de {high_nan_threshold*100:.0f}% de NaN.")

Type de données initial de X_train: object
X_train est de type 'object'. Tentative de conversion en np.float64...
Conversion de X_train en np.float64 réussie. Nouveau dtype: float64

=== ANALYSE DE LA QUALITÉ DES DONNÉES ===
Valeurs NaN dans X_train: 48050049
Valeurs infinies dans X_train: 0
Valeurs négatives (potentiellement problématiques): 0

Statistiques descriptives par feature (calculées sur les valeurs non-NaN pour mean/std/min/max):
   feature_idx        feature_name         mean          std        min           max  nan_rate  zero_rate
0            0          creatinine    10.293623    61.165592   0.000000   9532.000000  0.943660   0.000002
1            1                urea     0.661896    25.556834   0.000000   2869.086667  0.980429   0.000002
2            2  c_reactive_protein  1722.509578  3639.069854   0.000000   9532.000000  0.962918   0.000002
3            3    prothrombin_time    91.968974   248.683332  13.000000  21642.000000  0.988286   0.000000
4            4      

In [ ]:

# ## 3. Feature Engineering et Extraction de Features Temporelles

# --- MODIFICATION HERE ---
def extract_temporal_features(X, original_feature_names_list=None):
    """
    Extrait des features temporelles à partir des séries de 12 mois
    Utilise original_feature_names_list pour nommer les features dérivées.
    """
    n_samples, n_timesteps, n_features = X.shape

    # Initialisation des features extraites
    features_list = []
    engineered_feature_names_list = [] # Renamed for clarity

    for feat_idx in range(n_features):
        # Use original feature name if available, otherwise use index
        base_feat_name = f"origfeat_{feat_idx}"
        if original_feature_names_list is not None and feat_idx < len(original_feature_names_list):
            base_feat_name = original_feature_names_list[feat_idx]
        # Sanitize feature name for use in column names (e.g. remove spaces, special chars)
        base_feat_name = base_feat_name.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace("-", "_")


        # Données pour cette feature
        feat_data = X[:, :, feat_idx]  # Shape: (n_samples, 12)

        # 1. Statistiques de base
        features_list.append(np.nanmean(feat_data, axis=1))  # Moyenne sur 12 mois, ignorant les NaN
        engineered_feature_names_list.append(f"{base_feat_name}_mean")

        features_list.append(np.nanstd(feat_data, axis=1))   # Écart-type, ignorant les NaN
        engineered_feature_names_list.append(f"{base_feat_name}_std")

        features_list.append(np.nanmin(feat_data, axis=1))   # Minimum, ignorant les NaN
        engineered_feature_names_list.append(f"{base_feat_name}_min")

        features_list.append(np.nanmax(feat_data, axis=1))   # Maximum, ignorant les NaN
        engineered_feature_names_list.append(f"{base_feat_name}_max")

        # 2. Tendance (pente de régression linéaire)
        slopes = []
        x_reg = np.arange(n_timesteps)
        for i in range(n_samples):
            y_reg = feat_data[i]
            valid_indices = ~np.isnan(y_reg)
            if np.sum(valid_indices) >= 2: # Need at least 2 points for linregress
                try:
                    # stats.linregress can't handle NaNs directly
                    slope, _, _, _, _ = stats.linregress(x_reg[valid_indices], y_reg[valid_indices])
                    slopes.append(slope)
                except ValueError: # Catch potential errors if data is still problematic
                    slopes.append(np.nan) # Use np.nan for failed calculations
            else:
                slopes.append(np.nan) # Use np.nan if not enough data points

        features_list.append(np.array(slopes))
        engineered_feature_names_list.append(f"{base_feat_name}_slope")

        # 3. Coefficient de variation (handle potential division by zero or nan mean)
        mean_val = np.nanmean(feat_data, axis=1)
        std_val = np.nanstd(feat_data, axis=1)
        cv = np.divide(std_val, mean_val + 1e-8, out=np.full_like(mean_val, np.nan), where=mean_val!=0) # Add 1e-8 to mean for stability
        features_list.append(cv)
        engineered_feature_names_list.append(f"{base_feat_name}_cv")

        # 4. Différence entre derniers et premiers 3 mois
        # Use nanmean to be robust to NaNs within the 3-month windows
        first_3 = np.nanmean(feat_data[:, :3], axis=1)
        last_3 = np.nanmean(feat_data[:, -3:], axis=1)
        delta = last_3 - first_3
        features_list.append(delta)
        engineered_feature_names_list.append(f"{base_feat_name}_delta_3m")

        # 5. Moyenne mobile 3 mois (variance of MA)
        ma3_std_values = []
        for i in range(n_samples):
            patient_ma3_series = []
            # Calculate rolling mean if enough non-NaN points
            for t in range(n_timesteps - 2): # iterate up to the point where a 3-month window can start
                window = feat_data[i, t:t+3]
                if np.sum(~np.isnan(window)) > 0: # if at least one non-NaN value in window
                    patient_ma3_series.append(np.nanmean(window))
                else:
                    patient_ma3_series.append(np.nan)

            if len(patient_ma3_series) > 1 and np.sum(~np.isnan(patient_ma3_series)) > 1:
                 ma3_std_values.append(np.nanstd(patient_ma3_series))
            else:
                ma3_std_values.append(np.nan)

        features_list.append(np.array(ma3_std_values))
        engineered_feature_names_list.append(f"{base_feat_name}_ma3_std")
        
        # 6. Last known value (often very predictive)
        last_values = []
        for i in range(n_samples):
            patient_series = feat_data[i, :]
            valid_lv = patient_series[~np.isnan(patient_series)]
            if len(valid_lv) > 0:
                last_values.append(valid_lv[-1])
            else:
                last_values.append(np.nan)
        features_list.append(np.array(last_values))
        engineered_feature_names_list.append(f"{base_feat_name}_last_value")

        # 7. Number of non-NaN measurements
        features_list.append(np.sum(~np.isnan(feat_data), axis=1))
        engineered_feature_names_list.append(f"{base_feat_name}_num_measurements")


In [ ]:

    # Conversion en array
    X_engineered = np.column_stack(features_list)

    print(f"Features originales: {n_features}")
    print(f"Features extraites par feature originale: {X_engineered.shape[1] // n_features if n_features > 0 else 0}")
    print(f"Total features extraites: {X_engineered.shape[1]}")

    return X_engineered, engineered_feature_names_list
# --- END OF MODIFICATION ---


In [14]:

# Extraction des features temporelles
print("\n=== FEATURE ENGINEERING ===")
# Pass the original feature names to the function
X_train_engineered, engineered_feature_names = extract_temporal_features(X_train, feature_names_original)
X_eval_engineered, _ = extract_temporal_features(X_eval, feature_names_original) # Use same names for consistency

print(f"Shape après feature engineering (entraînement): {X_train_engineered.shape}")
print(f"Shape après feature engineering (évaluation): {X_eval_engineered.shape}")
print(f"Quelques noms de features ingéniérées: {engineered_feature_names[:10]}")


# Gestion des valeurs manquantes et normalisation
from sklearn.impute import SimpleImputer

print("\n=== PREPROCESSING ===")

# Imputation des valeurs manquantes
# Using median is generally robust. Consider different strategies or feature-specific imputation.
imputer = SimpleImputer(strategy='median') # Using median for robustness to outliers
X_train_imputed = imputer.fit_transform(X_train_engineered)
X_eval_imputed = imputer.transform(X_eval_engineered)

# Normalisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_eval_scaled = scaler.transform(X_eval_imputed)

print(f"Données après preprocessing (entraînement): {X_train_scaled.shape}")
print(f"Valeurs manquantes après imputation (entraînement): {np.isnan(X_train_scaled).sum()}")
print(f"Valeurs infinies après imputation (entraînement): {np.isinf(X_train_scaled).sum()}")


=== FEATURE ENGINEERING ===
Features originales: 77
Features extraites par feature originale: 10
Total features extraites: 770


TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [ ]:


# ## 4. Analyse de l'importance des features
def analyze_feature_importance(X, y, current_feature_names, n_top=20):
    """
    Analyse l'importance des features avec plusieurs méthodes
    """
    print("\n=== ANALYSE D'IMPORTANCE DES FEATURES ===")
    # 1. Test univarié (f_classif)
    print("--- Analyse Univariée (f_classif) ---")
    # Ensure no NaN/inf values are passed to SelectKBest
    if np.any(np.isnan(X)) or np.any(np.isinf(X)):
        print("Warning: NaN or Inf values detected in X before SelectKBest. This can cause errors.")
        # A more robust solution would be to re-impute or handle them here.
        # For now, we assume previous imputation handled this.
        
    selector_univariate = SelectKBest(f_classif, k='all')
    selector_univariate.fit(X, y)
    
    univariate_scores = selector_univariate.scores_
    univariate_pvalues = selector_univariate.pvalues_
    
    # Handle potential NaN scores/pvalues from f_classif (e.g., if a feature is constant)
    univariate_scores = np.nan_to_num(univariate_scores, nan=0.0)
    univariate_pvalues = np.nan_to_num(univariate_pvalues, nan=1.0) # Assign p-value of 1 to non-significant

    # 2. Random Forest Feature Importance
    print("--- Importance via Random Forest ---")
    rf_importance_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced')
    rf_importance_model.fit(X, y)
    rf_importance = rf_importance_model.feature_importances_
    
    # Création du DataFrame de résultats
    importance_df = pd.DataFrame({
        'feature_name': current_feature_names,
        'univariate_score': univariate_scores,
        'univariate_pvalue': univariate_pvalues,
        'rf_importance': rf_importance
    })
    
    # Normalisation des scores pour comparaison (MinMaxScaler like)
    for col in ['univariate_score', 'rf_importance']:
        min_val = importance_df[col].min()
        max_val = importance_df[col].max()
        if max_val == min_val: # Avoid division by zero if all values are the same
            importance_df[f'{col}_norm'] = 0.0 if max_val == 0 else 1.0
        else:
            importance_df[f'{col}_norm'] = (importance_df[col] - min_val) / (max_val - min_val)
    
    # Score composite (pondération ajustable)
    # Giving more weight to RF importance as it considers interactions.
    # Penalizing high p-values from univariate test.
    importance_df['composite_score'] = (
        importance_df['univariate_score_norm'] * 0.3 + # Weight for univariate F-score
        importance_df['rf_importance_norm'] * 0.7    # Weight for RF importance
        # (1 - importance_df['univariate_pvalue']) # Optional: factor in p-value directly
    )
    
    # Tri par score composite
    importance_df = importance_df.sort_values('composite_score', ascending=False).reset_index(drop=True)
    
    return importance_df

# Analyse de l'importance
# Make sure engineered_feature_names has the correct length
if X_train_scaled.shape[1] != len(engineered_feature_names):
    print(f"Warning: Mismatch in number of scaled features ({X_train_scaled.shape[1]}) and engineered feature names ({len(engineered_feature_names)}).")
    # Fallback if names are mismatched (should not happen if extract_temporal_features is correct)
    engineered_feature_names_corrected = [f"eng_feat_{i}" for i in range(X_train_scaled.shape[1])]
    importance_results = analyze_feature_importance(X_train_scaled, y_train, engineered_feature_names_corrected)
else:
    importance_results = analyze_feature_importance(X_train_scaled, y_train, engineered_feature_names)


# Affichage des top features
print("\n=== TOP 20 FEATURES (basé sur score composite) ===")
top_features_display = importance_results.head(20)
print(top_features_display[['feature_name', 'composite_score', 'univariate_score', 'rf_importance', 'univariate_pvalue']].to_string(index=False))

# Visualisation de l'importance des features
fig, axes = plt.subplots(1, 3, figsize=(22, 8)) # Changed to 1 row, 3 cols for better display
plt.subplots_adjust(wspace=0.4) # Add more space between plots

top_n_vis = 15 # Number of features to visualize

# Univariate F-score
top_univariate = importance_results.sort_values('univariate_score', ascending=False).head(top_n_vis)
axes[0].barh(range(len(top_univariate)), top_univariate['univariate_score'], color=sns.color_palette("viridis", top_n_vis))
axes[0].set_yticks(range(len(top_univariate)))
axes[0].set_yticklabels([name[:30] + '...' if len(name) > 30 else name for name in top_univariate['feature_name']], fontsize=8)
axes[0].set_title(f'Top {top_n_vis} Features - Univarié (F-score)')
axes[0].invert_yaxis()

# Random Forest Importance
top_rf = importance_results.sort_values('rf_importance', ascending=False).head(top_n_vis)
axes[1].barh(range(len(top_rf)), top_rf['rf_importance'], color=sns.color_palette("magma", top_n_vis))
axes[1].set_yticks(range(len(top_rf)))
axes[1].set_yticklabels([name[:30] + '...' if len(name) > 30 else name for name in top_rf['feature_name']], fontsize=8)
axes[1].set_title(f'Top {top_n_vis} Features - Random Forest')
axes[1].invert_yaxis()

# Composite Score
top_composite_vis = importance_results.head(top_n_vis) # Already sorted by composite_score
axes[2].barh(range(len(top_composite_vis)), top_composite_vis['composite_score'], color=sns.color_palette("cubehelix", top_n_vis))
axes[2].set_yticks(range(len(top_composite_vis)))
axes[2].set_yticklabels([name[:30] + '...' if len(name) > 30 else name for name in top_composite_vis['feature_name']], fontsize=8)
axes[2].set_title(f'Top {top_n_vis} Features - Score Composite')
axes[2].invert_yaxis()

plt.tight_layout()
plt.show()


In [ ]:

# ## 5. Sélection des meilleures features

# Sélection des top features pour la modélisation
# Can be based on a fixed number, a threshold on importance score, or p-value
n_selected_features = min(100, X_train_scaled.shape[1]) # Ensure not to select more features than available

# It's better to select features based on their original index in the scaled array,
# which corresponds to their order in `importance_results` if it was not re-indexed improperly.
# `importance_results` is already sorted by 'composite_score' descending.
# So, the indices of the top N features are simply the first N indices of the *original* feature set
# *if the importance_df was created with correct indices matching X_train_scaled columns*.

# The `importance_df` contains feature names. We need to map these names back to indices of X_train_scaled.
# `engineered_feature_names` holds the names in the order they appear as columns in `X_train_scaled`.
selected_feature_names_ordered = importance_results.head(n_selected_features)['feature_name'].tolist()

# Get the indices of these selected features from the original list of engineered feature names
# This ensures we select the correct columns from X_train_scaled
selected_features_indices = [engineered_feature_names.index(name) for name in selected_feature_names_ordered if name in engineered_feature_names]

# Check if all selected names were found
if len(selected_features_indices) != n_selected_features:
    print(f"Warning: Could only find {len(selected_features_indices)} out of {n_selected_features} selected feature names. This might indicate an issue.")
    # Fallback or error handling might be needed here. For now, proceed with found features.

X_train_selected = X_train_scaled[:, selected_features_indices]
X_eval_selected = X_eval_scaled[:, selected_features_indices]

print(f"\n=== SÉLECTION DE FEATURES ===")
print(f"Nombre de features sélectionnées: {X_train_selected.shape[1]}")
print(f"Shape finale pour entraînement: {X_train_selected.shape}")
print("Top 10 des features sélectionnées:")
for i, name in enumerate(selected_feature_names_ordered[:10]):
    print(f"  {i+1}. {name} (Composite Score: {importance_results.iloc[i]['composite_score']:.4f})")


# Analyse de corrélation entre top features (e.g., top 20 selected)
n_corr_features = min(20, X_train_selected.shape[1])
if n_corr_features > 1: # Need at least 2 features for correlation
    correlation_matrix = np.corrcoef(X_train_selected[:, :n_corr_features].T)

    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
                square=True, fmt='.2f', annot_kws={"size": 8}, cbar_kws={"shrink": .8})
    plt.title(f'Matrice de Corrélation - Top {n_corr_features} Features Sélectionnées')
    
    tick_labels = [name[:20] + '...' if len(name) > 20 else name for name in selected_feature_names_ordered[:n_corr_features]]
    plt.xticks(np.arange(n_corr_features) + 0.5, tick_labels, rotation=45, ha='right', fontsize=8)
    plt.yticks(np.arange(n_corr_features) + 0.5, tick_labels, rotation=0, fontsize=8)
    plt.tight_layout()
    plt.show()
else:
    print("Moins de 2 features sélectionnées, impossible d'afficher la matrice de corrélation.")


In [ ]:

# ## 6. Modélisation et évaluation

# Division train/validation avec stratification
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_selected, y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train # Important for imbalanced datasets
)

print(f"\n=== DIVISION DES DONNÉES POUR MODÉLISATION ===")
print(f"Train final: {X_train_final.shape}, Positifs: {np.mean(y_train_final)*100:.2f}%")
print(f"Validation: {X_val.shape}, Positifs: {np.mean(y_val)*100:.2f}%")

# --- MODEL TRAINING ---
models_dict = {} # To store trained models and their results

# 1. Random Forest
print("\n=== ENTRAÎNEMENT RANDOM FOREST ===")
rf_model = RandomForestClassifier(
    n_estimators=200,      # Number of trees
    max_depth=10,          # Max depth of trees
    min_samples_split=10,  # Min samples to split a node
    min_samples_leaf=5,    # Min samples in a leaf node
    class_weight='balanced', # Adjusts weights inversely proportional to class frequencies
    random_state=42,
    n_jobs=-1              # Use all available cores
)

rf_model.fit(X_train_final, y_train_final)
rf_pred_proba_val = rf_model.predict_proba(X_val)[:, 1] # Probabilities for the positive class
rf_pred_val = (rf_pred_proba_val > 0.5).astype(int) # Predictions based on 0.5 threshold

rf_auc_val = roc_auc_score(y_val, rf_pred_proba_val)
print(f"Random Forest AUC (Validation): {rf_auc_val:.4f}")
print("\nRandom Forest Classification Report (Validation):")
print(classification_report(y_val, rf_pred_val, target_names=['Non-Diabétique', 'Diabétique']))
print("\nConfusion Matrix (Validation):")
print(confusion_matrix(y_val, rf_pred_val))

models_dict['Random Forest'] = {'model': rf_model, 'proba_val': rf_pred_proba_val, 'auc_val': rf_auc_val}


# 2. Logistic Regression
print("\n=== ENTRAÎNEMENT LOGISTIC REGRESSION ===")
lr_model = LogisticRegression(
    class_weight='balanced', # Handles class imbalance
    random_state=42,
    solver='liblinear',      # Good for smaller datasets, handles L1/L2
    C=1.0,                   # Inverse of regularization strength
    max_iter=1000
)

lr_model.fit(X_train_final, y_train_final)
lr_pred_proba_val = lr_model.predict_proba(X_val)[:, 1]
lr_pred_val = (lr_pred_proba_val > 0.5).astype(int)

lr_auc_val = roc_auc_score(y_val, lr_pred_proba_val)
print(f"Logistic Regression AUC (Validation): {lr_auc_val:.4f}")
print("\nLogistic Regression Classification Report (Validation):")
print(classification_report(y_val, lr_pred_val, target_names=['Non-Diabétique', 'Diabétique']))
print("\nConfusion Matrix (Validation):")
print(confusion_matrix(y_val, lr_pred_val))

models_dict['Logistic Regression'] = {'model': lr_model, 'proba_val': lr_pred_proba_val, 'auc_val': lr_auc_val}
# ## 7. Évaluation comparative et courbes ROC

# Courbes ROC et Precision-Recall
plt.figure(figsize=(18, 6))

# ROC Curve
plt.subplot(1, 3, 1)
for model_name, results in models_dict.items():
    fpr, tpr, _ = roc_curve(y_val, results['proba_val'])
    plt.plot(fpr, tpr, label=f"{model_name} (AUC: {results['auc_val']:.3f})")

plt.plot([0, 1], [0, 1], 'k--', alpha=0.7, label='Aléatoire')
plt.xlabel('Taux de Faux Positifs (FPR)')
plt.ylabel('Taux de Vrais Positifs (TPR)')
plt.title('Courbes ROC (Validation)')
plt.legend()
plt.grid(True, alpha=0.4)

# Precision-Recall Curve
plt.subplot(1, 3, 2)
for model_name, results in models_dict.items():
    precision, recall, _ = precision_recall_curve(y_val, results['proba_val'])
    plt.plot(recall, precision, label=model_name)
no_skill = len(y_val[y_val==1]) / len(y_val)
plt.plot([0, 1], [no_skill, no_skill], 'k--', label='Pas de Compétence', alpha=0.7)
plt.xlabel('Rappel (Recall)')
plt.ylabel('Précision (Precision)')
plt.title('Courbes Précision-Rappel (Validation)')
plt.legend()
plt.grid(True, alpha=0.4)

# Distribution des probabilités prédites
plt.subplot(1, 3, 3)
for model_name, results in models_dict.items():
    sns.histplot(results['proba_val'][y_val == 0], bins=30, kde=True, stat="density", label=f'{model_name} (Négatifs)', alpha=0.5)
    sns.histplot(results['proba_val'][y_val == 1], bins=30, kde=True, stat="density", label=f'{model_name} (Positifs)', alpha=0.5)
plt.xlabel('Probabilité Prédite (Classe Positive)')
plt.ylabel('Densité')
plt.title('Distribution des Probabilités Prédites')
plt.legend(fontsize='small')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tableau de comparaison
comparison_list = []
for model_name, results in models_dict.items():
    comparison_list.append({'Modèle': model_name, 'AUC_Validation': results['auc_val']})

comparison_df = pd.DataFrame(comparison_list)
comparison_df = comparison_df.sort_values('AUC_Validation', ascending=False).reset_index(drop=True)

print("\n=== COMPARAISON DES MODÈLES (sur ensemble de validation) ===")
print(comparison_df.to_string(index=False))


In [ ]:

# ## 8. Génération des prédictions finales

# Sélection du meilleur modèle basé sur AUC de validation
if not comparison_df.empty:
    best_model_name = comparison_df.iloc[0]['Modèle']
    best_model_config = models_dict[best_model_name] # Contains the fitted model on X_train_final, y_train_final
    print(f"\n=== MEILLEUR MODÈLE (basé sur AUC validation): {best_model_name} (AUC: {best_model_config['auc_val']:.4f}) ===")

    # Ré-entraîner le meilleur modèle sur l'ensemble des données d'entraînement sélectionnées (X_train_selected, y_train)
    print(f"Réentraînement du modèle {best_model_name} sur toutes les données d'entraînement ({X_train_selected.shape[0]} échantillons)...")
    
    # Get the class and parameters of the best model
    # This assumes the model objects in models_dict store their original parameters
    # For sklearn models, get_params() is useful
    final_model = best_model_config['model'].__class__(**best_model_config['model'].get_params())
    
    # Ensure parameters like class_weight or scale_pos_weight are correctly set for the full dataset if they depend on it
    # For RandomForest and LogisticRegression with class_weight='balanced', it's handled internally.
    # For XGBoost, scale_pos_weight might need recalculation if used:
    # if best_model_name == 'XGBoost':
    #     full_scale_pos_weight = np.sum(y_train == 0) / np.sum(y_train == 1)
    #     final_model.set_params(scale_pos_weight=full_scale_pos_weight)

    final_model.fit(X_train_selected, y_train) # Fit on all X_train_selected and y_train
    
    # Prédictions sur les données d'évaluation
    final_predictions_proba = final_model.predict_proba(X_eval_selected)[:, 1]
else:
    print("Aucun modèle n'a été entraîné ou évalué. Impossible de générer les prédictions finales.")
    final_predictions_proba = np.zeros(len(sample_submission)) # Fallback

# Création du fichier de soumission
submission_df = sample_submission.copy()
submission_df['target'] = final_predictions_proba

# Sauvegarde
submission_filename = './submission_t2d_prediction_final.csv'
submission_df.to_csv(submission_filename, index=False)
print(f"\nPrédictions sauvegardées dans: {submission_filename}")
print(f"Statistiques des prédictions finales (probabilités):")
print(f"  - Min: {final_predictions_proba.min():.4f}")
print(f"  - Max: {final_predictions_proba.max():.4f}")
print(f"  - Moyenne: {final_predictions_proba.mean():.4f}")
print(f"  - Médiane: {np.median(final_predictions_proba):.4f}")
print(f"  - Std Dev: {np.std(final_predictions_proba):.4f}")

# Distribution des prédictions finales
plt.figure(figsize=(10, 6))
sns.histplot(final_predictions_proba, bins=50, kde=True, color='skyblue')
plt.xlabel('Probabilité Prédite (sur données d\'évaluation)')
plt.ylabel('Fréquence')
plt.title(f'Distribution des Prédictions Finales - {best_model_name if not comparison_df.empty else "N/A"}')
plt.grid(True, alpha=0.3)
plt.axvline(np.mean(final_predictions_proba), color='red', linestyle='--', label=f'Moyenne: {np.mean(final_predictions_proba):.3f}')
plt.axvline(np.median(final_predictions_proba), color='green', linestyle=':', label=f'Médiane: {np.median(final_predictions_proba):.3f}')
plt.legend()
plt.show()

print(f"\nPourcentage prédit comme positif avec seuil 0.5 (sur données d'évaluation): {np.mean(final_predictions_proba > 0.5)*100:.2f}%")


# ## 9. Résumé et recommandations

print("\n" + "=" * 60)
print("RÉSUMÉ DE L'ANALYSE")
print("=" * 60)
print(f"📊 Dataset: {X_train.shape[0]} patients × {X_train.shape[1]} mois × {X_train.shape[2]} features originales ({len(feature_names_original)} noms)")
print(f"🎯 Taux de cas positifs (entraînement): {np.mean(y_train)*100:.2f}%")
print(f"🔩 Features ingéniérées: {X_train_engineered.shape[1]} features ({len(engineered_feature_names)} noms)")
print(f"✅ Features sélectionnées pour modélisation: {X_train_selected.shape[1]}")
if not comparison_df.empty:
    print(f"🏆 Meilleur modèle (sur validation): {best_model_name} (AUC validation: {best_model_config['auc_val']:.4f})")
else:
    print("🏆 Meilleur modèle: Non déterminé.")
print(f"📈 Prédictions moyennes sur l'ensemble d'évaluation: {np.mean(final_predictions_proba):.3f}")

print("\n" + "=" * 60)
print("PISTES D'AMÉLIORATION")
print("=" * 60)
print("1. 🔬 Feature Engineering Approfondi:")
print("   - Interactions entre features (ex: ratio glucose/insuline si disponibles).")
print("   - Features basées sur des seuils cliniques (ex: nombre de fois où la glycémie > X).")
print("   - Utilisation de transformées (Fourier, ondelettes) pour capturer des périodicités.")
print("   - Features de type 'time since last abnormal value'.")
print("   - Traitement plus fin des valeurs manquantes (imputation multiple, modélisation des NaN).")

print("\n2. 🧠 Modélisation Avancée:")
print("   - Modèles d'ensemble plus sophistiqués (Stacking, Blending).")
print("   - Réseaux de neurones adaptés aux séquences (LSTM, GRU, Transformers) appliqués directement sur les données X_train (non-engineered).")
print("   - Modèles tenant compte de la structure temporelle de manière explicite lors de la validation croisée (ex: TimeSeriesSplit plus rigoureux).")

print("\n3. ⚙️ Optimisation et Calibration:")
print("   - Optimisation fine des hyperparamètres (ex: Optuna, Hyperopt, GridSearchCV avec K-Fold temporel).")
print("   - Optimisation du seuil de décision sur l'ensemble de validation pour maximiser une métrique métier (ex: F1-score, sensibilité à un rappel donné).")
print("   - Calibration des probabilités (ex: Isotonic Regression, Platt Scaling) pour que les scores de probabilité soient plus fiables.")

print("\n4. 📖 Interprétabilité et Analyse d'Erreurs:")
print("   - Utilisation de SHAP ou LIME pour comprendre les prédictions au niveau individuel et global.")
print("   - Analyse détaillée des faux positifs et faux négatifs: y a-t-il des profils de patients pour lesquels le modèle se trompe systématiquement?")
print("   - Évaluation de la robustesse du modèle face à de légères variations des données d'entrée.")

print("\n5. 💾 Gestion des Données et Pipeline:")
print("   - Mettre en place un pipeline ML robuste (ex: Scikit-learn Pipeline, MLflow) pour la reproductibilité et le déploiement.")
print("   - Versionnement des données et des modèles.")

print(f"\n✅ Analyse terminée! Fichier de soumission généré: {submission_filename}")